# < Insert Title here >

# Introduction
### Background 

Tennis is a popular, competitive sport played around the world. In tennis, players gain points by hitting a tennis ball, across a net in the court, in such a way that their opponent is unable to hit it back without the ball bouncing twice. 
Tennis can be played in "singles" where there is only one person on each side of the net or "doubles" where there are teams of two. 
It can be played on a variety of surfaces such as grass, clay, or hard court (i.e. like a gym floor)

- Professional tennis players participate in large **tournaments** for ranks, prizes, and fame. 
- In a tournament, players advance to the next round by winning their **match** against an opponent. 
- To win a match, a player must win two (or three, depending on the tournament) **sets**. 
- A set is won by the player who wins six **games**.
- A game is won by the first player to reach four **points**

The association of tennis professionals, or ATP, organizes these tournaments and also collects data on the players and the matches that take place.



### Our Question

### Our Dataset


# Preliminary Exploratory Data Analysis

# Methods

Each winner and loser of a match will have their stats mutated into three percentage predictors which will be used to Classify whether or not that player was the Winner or Loser of the match. The first predictor calculates successful saved breakpoints while the second and third predictors calculate serves and whether or not that Player won a point on their second serve or received a double fault.

We can apply these values by using scatterplot graphs to compare the Winner and Loser stats for each match and visualize a trend. For example, we can compare the amount of points won on the second serve versus the number of double faults and have each point colored on whether or not that player won or lost the match.

# Expected outcomes and significance